## 构建LDA模型

*1、去除句子停用词，用于生产句子分词*

In [49]:
'''
用于生成句子分词
'''
import re
import jieba as jb
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

# 对句子进行分词
def seg_sentence(sentence):
    sentence = re.sub(u'[0-9\.]+', u'', sentence)
    #jb.add_word('词汇')		# 这里是加入自定义的词来补充jieba词典
    sentence_seged = jb.cut(sentence.strip())
    stopwords = stopwordslist('/home/mw/input/soybean_policy2266/cn_stopwords.txt')  # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords and word.__len__()>1:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr


inputs = open(r"/home/mw/input/soybean_policy2266/大豆政策.txt", 'r', encoding='utf-8')

outputs = open(r"/home/mw/input/soybean_policy2266/大豆政策_分词.txt", 'w',encoding='utf-8')
for line in inputs:
    line_seg = seg_sentence(line)  # 这里的返回值是字符串
    outputs.write(line_seg + '\n')
outputs.close()
inputs.close()


<>:12: DeprecationWarning: invalid escape sequence '\.'
<>:12: DeprecationWarning: invalid escape sequence '\.'
/tmp/ipykernel_67/1713476716.py:12: DeprecationWarning: invalid escape sequence '\.'
  sentence = re.sub(u'[0-9\.]+', u'', sentence)


*2、构建LDA模型*

In [52]:
import codecs
from gensim import corpora
from gensim.models import LdaModel
from gensim.corpora import Dictionary


train = []

fp = codecs.open(r"/home/mw/input/soybean_policy2266/大豆政策_分词.txt",'r',encoding='utf8')
for line in fp:
    if line != '':
        line = line.split()
        train.append([w for w in line])

dictionary = corpora.Dictionary(train)

corpus = [dictionary.doc2bow(text) for text in train]

lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=4, passes=100)
# num_topics：主题数目
# passes：训练伦次
# num_words：每个主题下输出的term的数目

for topic in lda.print_topics(num_words = 20):
    termNumber = topic[0]
    print(topic[0], ':', sep='')
    listOfTerms = topic[1].split('+')
    for term in listOfTerms:
        listItems = term.split('*')
        print('  ', listItems[1], '(', listItems[0], ')', sep='')


0:
  "大豆" (0.048)
  "发展" ( 0.013)
  "市场" ( 0.013)
  "生产" ( 0.012)
  "玉米" ( 0.012)
  "进口" ( 0.010)
  "引导" ( 0.009)
  "加强" ( 0.009)
  "合理" ( 0.009)
  "种植" ( 0.009)
  "统筹" ( 0.007)
  "优势" ( 0.007)
  "区域" ( 0.007)
  "轮作" ( 0.007)
  "信息" ( 0.007)
  "资源" ( 0.006)
  "调减" ( 0.006)
  "实现" ( 0.006)
  "发布" ( 0.005)
  "方式"( 0.005)
1:
  "大豆" (0.038)
  "推进" ( 0.010)
  "生产" ( 0.010)
  "建设" ( 0.009)
  "发展" ( 0.009)
  "农田" ( 0.008)
  "农业" ( 0.008)
  "加强" ( 0.008)
  "加快" ( 0.007)
  "优势" ( 0.006)
  "供给" ( 0.006)
  "水平" ( 0.006)
  "国产" ( 0.006)
  "提高" ( 0.006)
  "高标准" ( 0.006)
  "发挥" ( 0.006)
  "成员" ( 0.006)
  "市场" ( 0.006)
  "地区" ( 0.006)
  "绿色"( 0.005)
2:
  "大豆" (0.062)
  "生产" ( 0.016)
  "玉米" ( 0.013)
  "政策" ( 0.013)
  "轮作" ( 0.012)
  "补贴" ( 0.011)
  "农民" ( 0.009)
  "目标" ( 0.009)
  "提高" ( 0.008)
  "种植" ( 0.008)
  "技术" ( 0.008)
  "面积" ( 0.008)
  "力争" ( 0.008)
  "振兴" ( 0.008)
  "高产" ( 0.008)
  "一批" ( 0.007)
  "公斤" ( 0.007)
  "开展" ( 0.007)
  "增产" ( 0.006)
  "实施"( 0.006)
3:
  "大豆" (0.053)
  "生产" ( 0.015)
  

*3、对划分好的主题进行可视化*

In [8]:
import codecs  
import pyLDAvis.gensim
from gensim import corpora  
from gensim.models import LdaModel  # 确保LdaModel的L是小写  
from gensim.corpora import Dictionary  
  
# 读取文本文件并构建语料库  
train = []  
with codecs.open(r"/home/mw/input/soybean_policy2266/大豆政策_分词.txt", 'r', encoding='utf8') as fp:  
    for line in fp:  
        if line.strip() != '':  # 使用strip()去除空白字符  
            train.append(line.split())  
  
# 构建字典和语料库  
dictionary = Dictionary(train)  
corpus = [dictionary.doc2bow(text) for text in train]  
  
# 训练LDA模型  
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=4, passes=10)  
  
# 打印主题和关键词  
for idx, topic in lda_model.print_topics(-1):  
    print('Topic: {} \nWords: {}'.format(idx, topic))  
  
# 准备LDA可视化  
# 这里不需要做任何修改，但为了消除警告，我们可能需要更新pyLDAvis的版本或修改相关代码  
lda_display = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)  
  
# 展示LDA可视化在notebook的output cell中  
pyLDAvis.display(lda_display)

/opt/conda/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


Topic: 0 
Words: 0.008*"贯彻" + 0.008*"十八" + 0.008*"精神" + 0.007*"战略" + 0.006*"总体" + 0.006*"发展" + 0.004*"统领" + 0.004*"五中全会" + 0.004*"四中" + 0.004*"产品安全"
Topic: 1 
Words: 0.073*"大豆" + 0.021*"生产" + 0.014*"发展" + 0.009*"资源" + 0.008*"提高" + 0.008*"加强" + 0.007*"引导" + 0.007*"市场" + 0.006*"力争" + 0.006*"进口"
Topic: 2 
Words: 0.042*"大豆" + 0.018*"玉米" + 0.017*"轮作" + 0.013*"种植" + 0.012*"实施" + 0.012*"开展" + 0.010*"黄淮海" + 0.010*"推进" + 0.009*"振兴" + 0.008*"农业"
Topic: 3 
Words: 0.043*"大豆" + 0.016*"补贴" + 0.016*"生产" + 0.014*"支持" + 0.014*"玉米" + 0.012*"种植" + 0.011*"政策" + 0.010*"发展" + 0.010*"稳定" + 0.010*"农民"


/opt/conda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
/opt/conda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


## OLS模型

In [3]:
#导入包
import pandas as pd
import statsmodels.api as sm
#读取文件
data = '/home/mw/input/soybean_zhibiao3569/确定指标.csv'
# 使用正确的编码读取CSV文件
df = pd.read_csv(data, encoding='gbk')
print(df)
print("---------------------------------------------")
endog = df['大豆单位面积产量(千克/公顷)']  # 因变量
exog = df[['成本利润率(%/亩)','净利润(元/亩)','每亩用工数量(日)',
           '土地成本(元/亩)','生产成本(元/亩)','人工成本(元/亩)',
           '农业生产指数(上年=100)','大豆进口数量(吨)','产值合计(元/亩)',
           '农村用电量(万千瓦时)','农作物受灾面积（千公顷）','乡村户数（万户）',
           '大豆播种面积(万公顷)','全国农业科技进步贡献率（%）']]  # 自变量

# 创建OLS模型并拟合
model = sm.OLS(endog, exog).fit()
print(model.summary())
print(model.ssr)


      年份  成本利润率(%/亩)  净利润(元/亩)  每亩化肥用量(公斤)  每亩种子用量(公斤)  农业机械总动力(万千瓦)  \
0   2007       42.92    122.92        9.11        4.80        2785.3   
1   2008       43.36    153.63        9.18        4.85        3018.4   
2   2009       22.10     85.06       10.05        4.79        3401.3   
3   2010       28.14    122.11       10.35        4.76        3736.3   
4   2011       20.99    105.81       10.19        4.58        4097.8   
5   2012       14.35     84.06       10.80        4.61        4549.3   
6   2013       -4.00    -25.31       10.76        4.90        4848.7   
7   2014        4.76     30.98       10.44        5.04        5155.5   
8   2015      -20.63   -137.71       10.08        5.11        5442.7   
9   2016      -38.78   -259.97       10.22        5.17        5634.3   
10  2017      -19.86   -130.52       10.32        5.06        5813.8   
11  2018      -29.28   -198.30       10.31        5.03        6082.4   
12  2019      -36.31   -263.55       10.35        4.98        52

/opt/conda/lib/python3.6/site-packages/scipy/stats/stats.py:1416: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  "anyway, n=%i" % int(n))


## 共线性

In [19]:
#用的是vif膨胀因子
import pandas as pd
import statsmodels.api as sm

# 创建数据
data = '/home/mw/input/soybean_zhibiao3569/确定指标.csv'
df = pd.read_csv(data, encoding='gbk')
print(df)
print("---------------------------------------------")
def vif(df, col_i):
    X = df.drop(col_i, axis=1)  # 保留除了当前列之外的所有列
    y = df[col_i]  # 当前列作为响应变量
    X = sm.add_constant(X)  # 添加截距项

    try:
        model = sm.OLS(y, X).fit()
        r2 = model.rsquared
        eps = 1e-9  # 用来避免除以零的小正数
        vif_value = 1. / (1. - r2 + eps)  # 加上 eps 防止除以零
        return vif_value
    except Exception as e:
        print(f"Error computing VIF for {col_i}: {e}")
        return None
# 选择计算VIF的列
test_data = df[[ '成本利润率(%/亩)','每亩用工数量(日)',
           '土地成本(元/亩)','生产成本(元/亩)','人工成本(元/亩)',
           '农业生产指数(上年=100)','大豆进口数量(吨)','产值合计(元/亩)',
           '农村用电量(万千瓦时)','农作物受灾面积（千公顷）','乡村户数（万户）',
           '大豆播种面积(万公顷)','全国农业科技进步贡献率（%）']]
#'成本利润率(%/亩)','净利润(元/亩)',
# 循环计算每一列的VIF
print("方差膨胀因子（VIF）：")
for i in test_data.columns:
    print(i, '\t', vif(df=test_data, col_i=i))


      年份  成本利润率(%/亩)  净利润(元/亩)  每亩化肥用量(公斤)  每亩种子用量(公斤)  农业机械总动力(万千瓦)  \
0   2007       42.92    122.92        9.11        4.80        2785.3   
1   2008       43.36    153.63        9.18        4.85        3018.4   
2   2009       22.10     85.06       10.05        4.79        3401.3   
3   2010       28.14    122.11       10.35        4.76        3736.3   
4   2011       20.99    105.81       10.19        4.58        4097.8   
5   2012       14.35     84.06       10.80        4.61        4549.3   
6   2013       -4.00    -25.31       10.76        4.90        4848.7   
7   2014        4.76     30.98       10.44        5.04        5155.5   
8   2015      -20.63   -137.71       10.08        5.11        5442.7   
9   2016      -38.78   -259.97       10.22        5.17        5634.3   
10  2017      -19.86   -130.52       10.32        5.06        5813.8   
11  2018      -29.28   -198.30       10.31        5.03        6082.4   
12  2019      -36.31   -263.55       10.35        4.98        52

Pearson相关系数

In [21]:
import pandas as pd
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
df = pd.read_csv('/home/mw/input/soybean_zhibiao3569/Pearson检验指标筛选.csv',encoding='gbk')
#计算相关系数
corr_matrix=df.corr()
#使用seaborn绘制热图
plt.rcParams['font.sans-serif']=['Source Han Sans CN']#使用黑体
plt.rcParams['axes.unicode_minus']=False#正确显示负号
plt.figure(figsize=(10,8))
sns.heatmap(corr_matrix,annot=True,cmap='coolwarm',linewidths=5,
            xticklabels=corr_matrix.columns,yticklabels=corr_matrix.columns)
plt.show()
#进行显著性检验
p_value,t_value=pearsonr(df['大豆单位面积产量(千克/公顷)'],df['全国农业科技进步贡献率（%）'])
'''
'成本利润率(%/亩)','每亩用工数量(日)',
'土地成本(元/亩)','生产成本(元/亩)','人工成本(元/亩)',
'农业生产指数(上年=100)','大豆进口数量(吨)','产值合计(元/亩)',
'农村用电量(万千瓦时)','农作物受灾面积（千公顷）','乡村户数（万户）',
'大豆播种面积(万公顷)','全国农业科技进步贡献率（%）'
'''
print("相关性系数：")
print(corr_matrix)
print("p_value：")
print(p_value)


<Figure size 720x576 with 2 Axes>

相关性系数：
                       年份  成本利润率(%/亩)  每亩用工数量(日)  土地成本(元/亩)  生产成本(元/亩)  \
年份               1.000000   -0.852068  -0.806658   0.963314   0.829561   
成本利润率(%/亩)      -0.852068    1.000000   0.817065  -0.792587  -0.762268   
每亩用工数量(日)       -0.806658    0.817065   1.000000  -0.835707  -0.722365   
土地成本(元/亩)        0.963314   -0.792587  -0.835707   1.000000   0.807082   
生产成本(元/亩)        0.829561   -0.762268  -0.722365   0.807082   1.000000   
人工成本(元/亩)        0.964849   -0.870746  -0.833583   0.942343   0.786001   
农业生产指数(上年=100)  -0.550036    0.548587   0.130469  -0.419942  -0.385583   
大豆进口数量(吨)        0.476247   -0.558023  -0.704491   0.616927   0.531947   
产值合计(元/亩)        0.357210    0.058721  -0.351365   0.510998   0.216717   
大豆单位面积产量(千克/公顷)  0.787287   -0.716495  -0.904797   0.806224   0.642656   
农村用电量(万千瓦时)      0.975951   -0.833736  -0.847817   0.956013   0.817839   
农作物受灾面积（千公顷）    -0.336693    0.137600   0.648621  -0.448283  -0.336498   
乡村户数（万户）         0.848371   -0.

## 因子分析

1、KMO检验，如果检验值低于0.5，证明不通过，则前面所处理的各个变量之间的相关性弱，共线性低

In [1]:
# 数据处理
from factor_analyzer.factor_analyzer import calculate_kmo
import pandas as pd
df = pd.read_csv('/home/mw/input/soybean_zhibiao3569/KMO检验指标.csv',encoding='gbk')
df.isnull().sum()

'''
KMO检验
检查变量间的相关性和偏相关性，取值在0-1之间；KOM统计量越接近1，
变量间的相关性越强，偏相关性越弱，因子分析的效果越好。
通常取值从0.6开始进行因子分析
'''
#KMO检验
kmo_all,kmo_model=calculate_kmo(df)
print("KMO检验为",kmo_model)

KMO检验为 0.7178545041204661


## elastic-net

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import ElasticNet, Lasso

df = pd.read_csv(r"/home/mw/input/soybean_zhibiao3569/确定指标.csv",encoding="gbk")
X=df.drop(columns=['大豆单位面积产量(千克/公顷)'])  # 特征  
y = df['大豆单位面积产量(千克/公顷)']    # 目标变量

1、使用scikit-learn拟合

In [3]:
from sklearn.linear_model import ElasticNet

# 初始化弹性网络回归器
reg = ElasticNet(alpha=0.1, l1_ratio=0.5, fit_intercept=False)
# 拟合线性模型
reg.fit(X, y)
# 权重系数
W = reg.coef_
W
# 打印特征名称和对应的权重系数
feature_names = X.columns
for feature, weight in zip(feature_names, W):
    print(f"Feature: {feature}, Weight: {weight}")

Feature: 成本利润率(%/亩), Weight: -13.291612273812861
Feature: 净利润(元/亩), Weight: 2.1662114169369167
Feature: 每亩化肥用量(公斤), Weight: 67.01421027055781
Feature: 每亩种子用量(公斤), Weight: 62.465120332938184
Feature: 每亩用工数量(日), Weight: -60.52180833428132
Feature: 人工成本(元/亩), Weight: 2.720548383439563
Feature: 农业生产指数(上年=100), Weight: 12.003098410978328
Feature: 乡村户数（万户）, Weight: -0.9307138927327461
Feature: 大豆播种面积(万公顷), Weight: 0.4440202568522585
Feature: 全国农业科技进步贡献率（%）, Weight: -6.816170498790709


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 31210.104818029715, tolerance: 4496.7349
  positive)


*可视化*

In [13]:
plt.rcParams['font.sans-serif']=['Source Han Sans CN']
# 设置正常显示符号
plt.rcParams["axes.unicode_minus"] = False
# 可视化特征重要性（条形图展示权重大小）
fig, ax = plt.subplots(figsize=(14, 8))  # 增大图表尺寸
index = np.arange(len(feature_names))  # 特征索引
bar_width = 0.35  # 设定条形图的宽度

# 绘制条形图
rects = ax.bar(index, W, bar_width, color='#457b9d', label='权重系数')

# 设置x轴标签
ax.set_xticks(index)
ax.set_xticklabels(feature_names, rotation=45, ha='right')  # 旋转x轴标签角度并设置对齐方式

# 设置图表标题和坐标轴标签
ax.set_title('ElasticNet 特征重要性', color='#264653')
ax.set_xlabel('特征', color='#264653')
ax.set_ylabel('权重系数', color='#264653')

# 调整坐标轴刻度标签颜色
ax.tick_params(axis='both', labelcolor='#264653')

# 显示图例
ax.legend()

# 显示图表
plt.show()


<Figure size 1008x576 with 1 Axes>

*计算权重系数*

*P对弹性网络回归的影响*

In [5]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
from sklearn.linear_model import ElasticNet  
plt.rcParams['font.sans-serif']=['Source Han Sans CN']
# 设置正常显示符号
plt.rcParams["axes.unicode_minus"] = False
# 加载数据  
df = pd.read_excel("/home/mw/input/soybean_zhibiao3569/Elastic-net检验.xls", encoding="gbk")  
X = df.drop(columns=['大豆单位面积产量(千克/公顷)','年份'])  # 特征  
y = df['大豆单位面积产量(千克/公顷)']  # 目标变量  
  
# 定义alpha值的范围  
alphas = np.logspace(-4, 2, 10)  # 从0.0001到100的等比数列  
  
# 初始化用于存储系数的数组  
coefs = np.zeros((len(alphas), X.shape[1]))  
  
# 拟合不同alpha值的模型，并存储系数  
for i, alpha in enumerate(alphas):  
    reg = ElasticNet(alpha=alpha, l1_ratio=0.5, fit_intercept=False)  
    reg.fit(X, y)  
    coefs[i] = reg.coef_  
  
# 绘制系数随alpha的变化  
plt.figure(figsize=(10, 6))  
for i, feature in enumerate(X.columns):  
    plt.plot(alphas, coefs[:, i], label=feature)  
  
plt.xscale('log')  
plt.xlabel('alpha')  
plt.ylabel('系数')  
plt.title('随着alpha 变化的弹性网络系数')  
plt.legend()  
plt.grid(True)  
plt.show()

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 17937.272764048856, tolerance: 4496.7349
  positive)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18128.852212569585, tolerance: 4496.7349
  positive)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18981.626559065342, tolerance: 4496.7349
  positive)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22271.992394899866, tolerance: 4496.7

<Figure size 720x432 with 1 Axes>

*弹性网络回归与Lasso对比*

In [9]:
import numpy as np  
import matplotlib.pyplot as plt 
import pandas as pd  
from sklearn.linear_model import ElasticNet, Lasso  
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['Source Han Sans CN']
# 设置正常显示符号
plt.rcParams["axes.unicode_minus"] = False
# 加载大豆数据  
df = pd.read_excel(r"/home/mw/input/soybean_zhibiao3569/Elastic-net检验.xls", encoding="gbk")  
X = df.drop(columns=['大豆单位面积产量(千克/公顷)','年份'])  # 特征  
y = df['大豆单位面积产量(千克/公顷)']  # 目标变量  
  
# 定义正则化强度的范围  
n_alphas = 100  
alphas = np.logspace(-3, 0.5, n_alphas)  
  
# 初始化ElasticNet和Lasso模型，并收集系数  
coefs_enet = []  
coefs_lasso = []  
  
for a in alphas:  
    # ElasticNet  
    reg_enet = ElasticNet(alpha=a, l1_ratio=0.9, fit_intercept=False)  
    reg_enet.fit(X, y)  
    coefs_enet.append(reg_enet.coef_)  
      
    # Lasso  
    reg_lasso = Lasso(alpha=a, fit_intercept=False)  
    reg_lasso.fit(X, y)  
    coefs_lasso.append(reg_lasso.coef_)  
  
# 准备绘制图像  
plt.figure(figsize=(14, 7))  
  
# 绘制ElasticNet系数随alpha变化的图像  
plt.subplot(1, 2, 1)  
for i, coef in enumerate(coefs_enet[0]):  # 只取第一个alpha值对应的系数进行展示  
    plt.plot(alphas, [c[i] for c in coefs_enet], label=X.columns[i])  
plt.xlabel('alpha')  
plt.ylabel('系数')  
plt.title('Elastic-Net系数')  
plt.xscale('log')  
plt.legend(loc='best', ncol=2)  # 调整图例位置，避免重叠  
  
# 绘制Lasso系数随alpha变化的图像  
plt.subplot(1, 2, 2)  
for i, coef in enumerate(coefs_lasso[0]):  # 只取第一个alpha值对应的系数进行展示  
    plt.plot(alphas, [c[i] for c in coefs_lasso], label=X.columns[i], linestyle='--')  
plt.xlabel('alpha')  
plt.ylabel('系数')  
plt.title('Lasso 系数')  
plt.xscale('log')  
plt.legend(loc='best', ncol=2)  # 调整图例位置，避免重叠  
  
# 突出显示重要特征（例如，基于系数绝对值的大小）  
# 这里您可以根据需要添加逻辑来突出显示特征  
  
# 调整子图之间的间距并显示图像  
plt.tight_layout()  
plt.show()

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 17996.419092196647, tolerance: 4496.7349
  positive)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 17892.005491248765, tolerance: 4496.7349
  positive)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18005.89712863896, tolerance: 4496.7349
  positive)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 17892.670957592014, tolerance: 4496.73

<Figure size 1008x504 with 2 Axes>

## R²检验

In [25]:
'''
对多模型权重系数检验
'''
import statsmodels.api as sm
import pandas as pd

# 读取数据
data = pd.read_csv(r"/home/mw/input/soybean_zhibiao3569/多模型筛选后.csv",encoding='gbk')

# 拟合回归模型
x = data.drop(columns=['大豆单位面积产量(千克/公顷)','年份'])  # 特征 
y = data['大豆单位面积产量(千克/公顷)']

X = sm.add_constant(x)
model = sm.OLS(y, X).fit()  # 注意这里使用X而不是x，并且fit()方法调用没有空格

# 显示调整后的R平方
print("经过OLS模型、VIF、Pearson相关系数处理，进行R²检验的值：")
print(model.rsquared_adj)  # 注意没有空格，并且前面有model.前缀


经过OLS模型、VIF、Pearson相关系数处理，进行R²检验的值：
0.8916363808155049


In [22]:
'''
Elastic-Net权重系数检验
'''
import statsmodels.api as sm
import pandas as pd

# 读取数据
data = pd.read_excel("/home/mw/input/soybean_zhibiao3569/Elastic-net检验.xls",encoding='gbk')

# 拟合回归模型
x = data.drop(columns=['大豆单位面积产量(千克/公顷)','年份'])  # 特征 
y = data['大豆单位面积产量(千克/公顷)']

X = sm.add_constant(x)
model = sm.OLS(y, X).fit()  # 注意这里使用X而不是x，并且fit()方法调用没有空格

# 显示调整后的R平方
print("经过Elastic-Net回归算法处理，进行R²检验的值：")
print(model.rsquared_adj)  # 注意没有空格，并且前面有model.前缀

经过Elastic-Net回归算法处理，进行R²检验的值：
0.9458706826994898
